<a href="https://colab.research.google.com/github/unaizanouman/zePOP/blob/main/zePOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update -y
!apt-get install -y mpich


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 https://cli.github.com/packages stable InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,153 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [6,008 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntug

In [ ]:
%%bash
cat << 'EOF' > graph_module.h
#ifndef GRAPH_MODULE_H
#define GRAPH_MODULE_H
#include <bits/stdc++.h>
using namespace std;

namespace graphmod {
    struct Graph {
        int n;
        vector<vector<int>> adj;
        vector<unordered_map<int,double>> delay;
    };

    inline Graph build_default_graph(int n){
        Graph g;
        g.n=n;
        g.adj.resize(n);
        g.delay.resize(n);
        mt19937_64 rng(12345);

        for(int i=0;i<n;i++){
            int a=(i+1)%n, b=(i-1+n)%n;
            g.adj[i].push_back(a);
            g.adj[i].push_back(b);
        }

        for(int i=0;i<n;i++){
            int extra=(i*7+13)%n;
            if(extra!=i){
                g.adj[i].push_back(extra);
                g.adj[extra].push_back(i);
            }
        }

        for(int i=0;i<n;i++){
            sort(g.adj[i].begin(), g.adj[i].end());
            g.adj[i].erase(unique(g.adj[i].begin(), g.adj[i].end()), g.adj[i].end());
        }

        for(int u=0;u<n;u++){
            for(int v:g.adj[u]){
                if(!g.delay[u].count(v)){
                    rng.seed(5000 + min(u,v)*10000 + max(u,v));
                    double d = 1.0 + (rng()%9000)/1000.0;
                    g.delay[u][v]=d;
                    g.delay[v][u]=d;
                }
            }
        }
        return g;
    }
}
#endif
EOF

cat << 'EOF' > zepop_library.h
#ifndef ZEPOP_LIBRARY_H
#define ZEPOP_LIBRARY_H
#include <mpi.h>
#include <bits/stdc++.h>
#include "graph_module.h"
using namespace std;
using namespace graphmod;

namespace zepop {
    const double INF = 1e18;

    inline vector<double> compute_distances(const Graph &G,int rank,int n){
        vector<double> dist(n,INF);
        dist[rank]=0;

        for(auto &p:G.delay[rank]) dist[p.first]=p.second;

        for(int r=0;r<n+5;r++){
            vector<double> sendbuf=dist;
            for(int nb:G.adj[rank])
                MPI_Send(sendbuf.data(),n,MPI_DOUBLE,nb,0,MPI_COMM_WORLD);

            bool changed=false;
            for(int nb:G.adj[rank]){
                vector<double> recv(n);
                MPI_Recv(recv.data(),n,MPI_DOUBLE,nb,0,MPI_COMM_WORLD,MPI_STATUS_IGNORE);
                double w=G.delay[rank].at(nb);
                for(int k=0;k<n;k++){
                    double cand=w+recv[k];
                    if(cand<dist[k]){
                        dist[k]=cand;
                        changed=true;
                    }
                }
            }
            int l=changed?1:0, g=0;
            MPI_Allreduce(&l,&g,1,MPI_INT,MPI_SUM,MPI_COMM_WORLD);
            if(g==0) break;
        }
        return dist;
    }

    inline vector<double> gather_all_dist(const vector<double>&dist,int n){
        vector<double> all(n*n);
        MPI_Allgather(dist.data(),n,MPI_DOUBLE,all.data(),n,MPI_DOUBLE,MPI_COMM_WORLD);
        return all;
    }

    inline int elect_leader(const vector<double>&C){
        double best=-1;
        int leader=-1;
        for(int i=0;i<C.size();i++){
            if(C[i]>best){
                best=C[i];
                leader=i;
            }
        }
        return leader;
    }
}
#endif
EOF

cat << 'EOF' > main_zepop_code.cpp
#include <mpi.h>
#include <bits/stdc++.h>
#include "graph_module.h"
#include "zepop_library.h"
using namespace std;
using namespace graphmod;
using namespace zepop;

int main(int argc,char**argv){
    MPI_Init(&argc,&argv);
    int rank,size;
    MPI_Comm_rank(MPI_COMM_WORLD,&rank);
    MPI_Comm_size(MPI_COMM_WORLD,&size);

    Graph G = build_default_graph(size);

    vector<double> dist = compute_distances(G,rank,size);
    vector<double> allD = gather_all_dist(dist,size);

    auto D=[&](int u,int v){ return allD[u*size+v]; };

    double Tx=0;
    for(int i=0;i<size;i++) if(i!=rank) Tx+=D(rank,i);
    double closeness=(size>1)?(double)(size-1)/Tx : 0;

    vector<double> allC(size);
    MPI_Allgather(&closeness,1,MPI_DOUBLE,allC.data(),1,MPI_DOUBLE,MPI_COMM_WORLD);

    int leader = elect_leader(allC);

    int parent=-1;
    if(rank!=leader){
        double best=D(rank,leader);
        int choice=-1;
        for(int nb:G.adj[rank]){
            double w=G.delay[rank].at(nb);
            if(fabs((w + D(nb,leader)) - best) < 1e-6){
                if(choice==-1 || nb<choice) choice=nb;
            }
        }
        parent=choice;
    }

    vector<int> allP(size);
    MPI_Allgather(&parent,1,MPI_INT,allP.data(),1,MPI_INT,MPI_COMM_WORLD);

    if(rank==0){
        cout<<"=== ZePoP Clean Modular Output ===\n";
        cout<<"Leader: "<<leader<<"\n";
        for(int i=0;i<size;i++){
            cout<<"Node "<<i<<" -> Parent: "<<allP[i]<<"\n";
        }
    }

    MPI_Finalize();
    return 0;
}
EOF


In [ ]:
!mpicxx -std=c++11 -O2 main_zepop_code.cpp -o zepop


In [ ]:
!mpirun --allow-run-as-root --oversubscribe -np 8 ./zepop


=== ZePoP Clean Modular Output ===
Leader: 0
Node 0 -> Parent: -1
Node 1 -> Parent: 0
Node 2 -> Parent: 1
Node 3 -> Parent: 4
Node 4 -> Parent: 5
Node 5 -> Parent: 0
Node 6 -> Parent: 7
Node 7 -> Parent: 0


In [2]:
# Install MPI and python plotting libs
!apt-get update -y
!apt-get install -y mpich
!pip install networkx matplotlib numpy


Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [3]:
%%bash
cat << 'EOF' > graph_module.h
#ifndef GRAPH_MODULE_H
#define GRAPH_MODULE_H
#include <bits/stdc++.h>
using namespace std;

// Simple header-only graph builder used by ZePoP demo.
namespace graphmod {
    struct Graph {
        int n;
        vector<vector<int>> adj;
        vector<unordered_map<int,double>> delay; // symmetric delays
    };

    inline Graph build_default_graph(int n){
        Graph g;
        g.n = n;
        g.adj.resize(n);
        g.delay.resize(n);
        mt19937_64 rng(12345);

        for(int i=0;i<n;i++){
            int a = (i+1)%n;
            int b = (i-1+n)%n;
            g.adj[i].push_back(a);
            g.adj[i].push_back(b);
        }
        // Add one extra deterministic edge per node to increase connectivity
        for(int i=0;i<n;i++){
            int extra = (i*7 + 13) % n;
            if(extra != i){
                g.adj[i].push_back(extra);
                g.adj[extra].push_back(i);
            }
        }
        // Normalize adjacency: sort + unique
        for(int i=0;i<n;i++){
            sort(g.adj[i].begin(), g.adj[i].end());
            g.adj[i].erase(unique(g.adj[i].begin(), g.adj[i].end()), g.adj[i].end());
        }

        // Generate deterministic symmetric link delays between 1.0 and 10.0
        for(int u=0; u<n; ++u){
            for(int v: g.adj[u]){
                if(g.delay[u].count(v)==0){
                    uint64_t seed = 5000 + (uint64_t)min(u,v)*10000 + (uint64_t)max(u,v);
                    rng.seed(seed);
                    double d = 1.0 + (double)(rng() % 9000) / 1000.0; // [1.0,10.0)
                    g.delay[u][v] = d;
                    g.delay[v][u] = d;
                }
            }
        }
        return g;
    }
}
#endif
EOF

cat << 'EOF' > zepop_library.h
#ifndef ZEPOP_LIBRARY_H
#define ZEPOP_LIBRARY_H
#include <mpi.h>
#include <bits/stdc++.h>
#include "graph_module.h"
using namespace std;
using namespace graphmod;

namespace zepop {
    const double INF = 1e18;

    // Distributed distance computation (distance-vector style).
    inline vector<double> compute_distances(const Graph &g, int rank, int size){
        int n = size;
        vector<double> dist(n, INF);
        dist[rank] = 0.0;
        for(auto &p: g.delay[rank]) dist[p.first] = p.second;

        int max_rounds = n + 5;
        for(int round=0; round<max_rounds; ++round){
            vector<double> sendbuf = dist;
            for(int nb: g.adj[rank]){
                MPI_Send(sendbuf.data(), n, MPI_DOUBLE, nb, 0, MPI_COMM_WORLD);
            }
            bool local_changed = false;
            for(int nb: g.adj[rank]){
                vector<double> recv(n);
                MPI_Recv(recv.data(), n, MPI_DOUBLE, nb, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
                double w = g.delay[rank].at(nb);
                for(int k=0;k<n;k++){
                    if(recv[k] >= INF/2) continue;
                    double cand = w + recv[k];
                    if(cand + 1e-9 < dist[k]){
                        dist[k] = cand;
                        local_changed = true;
                    }
                }
            }
            int l = local_changed ? 1 : 0;
            int gchg = 0;
            MPI_Allreduce(&l, &gchg, 1, MPI_INT, MPI_SUM, MPI_COMM_WORLD);
            if(gchg == 0) break;
        }
        return dist;
    }

    inline vector<double> gather_all_dist(const vector<double> &dist, int n){
        vector<double> allD(n * n);
        MPI_Allgather(dist.data(), n, MPI_DOUBLE, allD.data(), n, MPI_DOUBLE, MPI_COMM_WORLD);
        return allD;
    }

    inline int elect_leader(const vector<double> &allC){
        int n = (int)allC.size();
        int leader = -1;
        double best = -1.0;
        for(int i=0;i<n;i++){
            double c = allC[i];
            if(c > best + 1e-12){
                best = c; leader = i;
            } else if(fabs(c - best) < 1e-12 && c > -0.5){
                if(i < leader) leader = i;
            }
        }
        return leader;
    }
}
#endif
EOF

cat << 'EOF' > main_zepop_code.cpp
#include <mpi.h>
#include <bits/stdc++.h>
#include "graph_module.h"
#include "zepop_library.h"
using namespace std;
using namespace graphmod;
using namespace zepop;

int main(int argc, char** argv){
    MPI_Init(&argc, &argv);
    int rank,size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    int n = size;

    // Build deterministic default graph (same view at all processes)
    Graph G = build_default_graph(n);

    // 1) Distributed compute distances
    vector<double> dist = compute_distances(G, rank, n);

    // 2) Gather full distance matrix (rows: source u, columns: dest v; D[u][v])
    vector<double> allD = gather_all_dist(dist, n);

    auto D = [&](int u,int v)->double{ return allD[u*n + v]; };

    // 3) Compute closeness centrality for this node (delay-based)
    // closeness_x = (n-1) / sum_{s != x} D[s][x]  [distance from s to x]
    // note: allD[u*n + v] is distance from u -> v, so we need column sums
    double sum_from_others_to_me = 0.0;
    for(int s=0;s<n;s++){
        if(s==rank) continue;
        double dsx = D(s, rank);
        if(dsx >= INF/2) dsx = 1e9;
        sum_from_others_to_me += dsx;
    }
    double closeness = (n>1) ? ((double)(n-1) / sum_from_others_to_me) : 0.0;

    // 4) Compute next-hop from this node to every destination (for Ixy)
    vector<int> next_hop(n, -1);
    for(int s=0;s<n;s++){
        if(s==rank) continue;
        double best = D(rank, s);
        int choice = -1;
        for(int y: G.adj[rank]){
            double w = G.delay[rank].at(y);
            double dy_s = D(y, s);
            if(dy_s >= INF/2) continue;
            if ( fabs((w + dy_s) - best) < 1e-6 ){
                if(choice==-1 || y < choice) choice = y;
            }
        }
        next_hop[s] = choice;
    }

    // 5) Compute Ixy (local) and gather to compute Oxy
    vector<int> localI(n, 0);
    for(int s=0;s<n;s++){
        if(s==rank) continue;
        int nh = next_hop[s];
        if(nh != -1) localI[nh] += 1;
    }
    vector<int> allI(n * n, 0);
    MPI_Allgather(localI.data(), n, MPI_INT, allI.data(), n, MPI_INT, MPI_COMM_WORLD);

    unordered_map<int,int> Ixy, Oxy;
    for(int y: G.adj[rank]){ Ixy[y] = localI[y]; Oxy[y] = allI[y*n + rank]; }

    // 6) Determine phi[x,y] for each neighbor y
    unordered_map<int,bool> phi;
    for(int y: G.adj[rank]){
        int ixy = Ixy[y];
        int oxy = Oxy[y];
        int delta = oxy - ixy;
        // compute Ty (sum delays from nodes in C to y) - we approximate by total distances to y
        double Ty = 0.0;
        for(int s=0;s<n;s++){
            if(s==y) continue;
            double d = D(s, y);
            if(d >= INF/2) d = 1e9;
            Ty += d;
        }
        bool ok=false;
        if(delta > 0) ok = true;
        else if(delta < 0) ok = false;
        else{
            if(fabs(sum_from_others_to_me - Ty) > 1e-6) ok = (sum_from_others_to_me < Ty);
            else ok = (rank < y);
        }
        phi[y] = ok;
    }

    // 7) Node is candidate if phi[y] true for all neighbors
    bool candidacy = true;
    for(int y: G.adj[rank]) if(!phi[y]) { candidacy=false; break; }

    // 8) Broadcast candidacy closeness (non-candidates send -1)
    double myC = candidacy ? closeness : -1.0;
    vector<double> allC(n, -1.0);
    MPI_Allgather(&myC, 1, MPI_DOUBLE, allC.data(), 1, MPI_DOUBLE, MPI_COMM_WORLD);

    // 9) Elect leader (highest closeness, tie-break lower id)
    int leader = elect_leader(allC);

    // fallback if no candidate
    if(leader == -1){
        // pick node with minimum sum_from_others (i.e., best average)
        vector<double> allSum(n, 0.0);
        MPI_Allgather(&sum_from_others_to_me,1,MPI_DOUBLE, allSum.data(),1,MPI_DOUBLE, MPI_COMM_WORLD);
        double bestVal = 1e300; int bestId = 0;
        for(int u=0;u<n;u++){
            if(allSum[u] < bestVal){ bestVal = allSum[u]; bestId = u; }
        }
        leader = bestId;
    }

    // 10) Build parent (DCDT): neighbor on shortest path to leader
    int parent = -1;
    if(rank == leader) parent = -1;
    else {
        double best = D(rank, leader);
        int choice = -1;
        for(int y: G.adj[rank]){
            double w = G.delay[rank].at(y);
            double dy = D(y, leader);
            if(dy >= INF/2) continue;
            if(fabs((w + dy) - best) < 1e-6){
                if(choice == -1 || y < choice) choice = y;
            }
        }
        parent = choice;
    }
    // gather parents
    vector<int> all_parents(n, -2);
    MPI_Allgather(&parent,1,MPI_INT, all_parents.data(),1,MPI_INT, MPI_COMM_WORLD);

    // Rank 0 writes files for visualization and prints summary
    if(rank==0){
        // write edges file (u v weight)
        FILE *fe = fopen("graph_edges.txt","w");
        for(int u=0;u<n;u++){
            for(int v: G.adj[u]){
                if(u < v){ // write each undirected edge once
                    fprintf(fe, "%d %d %.6f\n", u, v, G.delay[u].at(v));
                }
            }
        }
        fclose(fe);

        // write distance matrix (rows u: distances from u to v)
        FILE *fd = fopen("distance_matrix.txt","w");
        for(int u=0;u<n;u++){
            for(int v=0;v<n;v++){
                fprintf(fd, "%.6f%s", allD[u*n + v], (v+1==n? "": " "));
            }
            fprintf(fd, "\n");
        }
        fclose(fd);

        // write closeness per node (as computed by algorithm and candidacy)
        FILE *fc = fopen("closeness.txt","w");
        for(int i=0;i<n;i++){
            fprintf(fc, "%d %.12f\n", i, allC[i]);
        }
        fclose(fc);

        // write parents
        FILE *fp = fopen("parents.txt","w");
        for(int i=0;i<n;i++){
            fprintf(fp, "%d %d\n", i, all_parents[i]);
        }
        fclose(fp);

        // write chosen leader
        FILE *fl = fopen("leader.txt","w");
        fprintf(fl, "%d\n", leader);
        fclose(fl);

        // Print quick human summary
        cout<<"=== ZePoP MPI Output (files written) ===\n";
        cout<<"Leader: "<<leader<<"\n";
        cout<<"Files written: graph_edges.txt, distance_matrix.txt, closeness.txt, parents.txt, leader.txt\n";
        cout<<"\nCloseness (candidates show value; non-candidates -1):\n";
        for(int i=0;i<n;i++){
            if(allC[i] < -0.5) printf(" node %2d : %7s\n", i, "-");
            else printf(" node %2d : %0.6f\n", i, allC[i]);
        }
        cout<<"\nParent list:\n";
        for(int i=0;i<n;i++){
            printf(" node %2d -> parent = %d\n", i, all_parents[i]);
        }
    }

    MPI_Finalize();
    return 0;
}
EOF


In [4]:
!mpicxx -std=c++11 -O2 main_zepop_code.cpp -o zepop


In [5]:
!mpirun --allow-run-as-root --oversubscribe -np 8 ./zepop


=== ZePoP MPI Output (files written) ===
Leader: 4
Files written: graph_edges.txt, distance_matrix.txt, closeness.txt, parents.txt, leader.txt

Closeness (candidates show value; non-candidates -1):
 node  0 :       -
 node  1 :       -
 node  2 :       -
 node  3 :       -
 node  4 : 0.161711
 node  5 :       -
 node  6 :       -
 node  7 :       -

Parent list:
 node  0 -> parent = 5
 node  1 -> parent = 4
 node  2 -> parent = 1
 node  3 -> parent = 4
 node  4 -> parent = -1
 node  5 -> parent = 4
 node  6 -> parent = 5
 node  7 -> parent = 0
